

# import pretrained sentiment model



In [1]:



from transformers import pipeline
classifier = pipeline(
                      task="zero-shot-classification",
#                       device=0,
                      model="facebook/bart-large-mnli"
                    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

NameError: name 'text' is not defined

In [18]:
import warnings
import pandas as pd

# test it on some phrase


In [65]:

classifier(" How Texans' D has evolved under Crennel Vrabel ",["positive","negative",'neutral'],multi_class=True)



The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


{'sequence': " How Texans' D has evolved under Crennel Vrabel ",
 'labels': ['positive', 'negative', 'neutral'],
 'scores': [0.9336717128753662, 0.0135049382224679, 0.003030309220775962]}

# next 3 blocks are building a function to get the total sentiment score


In [66]:
output = classifier("How Texans' D has evolved under Crennel Vrabel",["positive","negative",'neutral'],multi_class=True)


The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [67]:
positive = output['scores'][output['labels'].index('positive')]
negative = output['scores'][output['labels'].index('negative')]
total= positive - negative
print("Positive Score:", total)

Positive Score: 0.8488707710057497


In [80]:

def get_total_score(text):
    labels = ["positive", "negative", "neutral"]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        output = classifier(text, labels, multi_label=True)
    positive_score = output['scores'][output['labels'].index('positive')]
    negative_score = output['scores'][output['labels'].index('negative')]
    neutral_score = output['scores'][output['labels'].index('neutral')]

    if (neutral_score)>0.50:
        total_score = 0
    else:
        total_score = positive_score - negative_score


    return total_score

In [69]:
# Example usage
text = "How Texans' D has evolved under Crennel Vrabel  "
total_score = get_total_score(text)
print("Total Score:", total_score)

Total Score: 0.8459009528160095


# Read and then clean up the data from the headlines

In [21]:
df = pd.read_csv("/content/sorted_df_w_teams_andDates3.csv")



# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7061 entries, 0 to 7060
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   archiveDate  7061 non-null   object
 1   Urls         6629 non-null   object
 2   headlines    6629 non-null   object
 3   team         6629 non-null   object
dtypes: object(4)
memory usage: 220.8+ KB


In [26]:

# Drop duplicates from the "headlines" column
df.drop_duplicates(subset="headlines", inplace=True)


df = df.dropna()

# # Display the resulting dataframe
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5378 entries, 19 to 7045
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   archiveDate  5378 non-null   object
 1   Urls         5378 non-null   object
 2   headlines    5378 non-null   object
 3   team         5378 non-null   object
dtypes: object(4)
memory usage: 210.1+ KB


In [76]:
df['headlines'] = df['headlines'].str.strip()


df.head(10)

,archiveDate,Urls,headlines,team
19,2017-08-20,http://web.archive.org/web/20170820000936/http...,Jags must make QB change; Cutler could have ca...,Jacksonville Jaguars
20,2017-08-20,http://web.archive.org/web/20170820000936/http...,How Texans' D has evolved under Crennel Vrabel,Houston Texans
21,2017-08-20,http://web.archive.org/web/20170820000936/http...,Cutler's debut proves he's in control of Dolph...,Miami Dolphins
22,2017-08-20,http://web.archive.org/web/20170820000936/http...,Raiders' Sean Smith charged with felony assault,Las Vegas Raiders
23,2017-08-20,http://web.archive.org/web/20170820000936/http...,Making sense of the Patriots fantasy RBs,New England Patriots
37,2017-08-22,http://web.archive.org/web/20170822000344/http...,"""Crazy"" ""weird"" eclipse rocks Titans' world",Tennessee Titans
38,2017-08-22,http://web.archive.org/web/20170822000344/http...,Beth Mowins to make history during Chargers-De...,Los Angeles Chargers
39,2017-08-22,http://web.archive.org/web/20170822000344/http...,Ravens embark on historic trip in lieu of prac...,Baltimore Ravens
43,2017-08-23,http://web.archive.org/web/20170823004000/http...,Mark Ingram and the Saints RBs in fantasy foot...,New Orleans Saints
44,2017-08-23,http://web.archive.org/web/20170823004000/http...,Niners assistant Sowers is first openly LGBT c...,San Francisco 49ers


In [120]:
# prompt: select 101-200 of df and call it df2

df2 = df.iloc[2201:2700]


# This is where I apply it to the data. But it takes a lot of computing power and is slow so I just did 10 right here. We will probably have to do it in chunks if we actually use this

In [121]:
df2['total_score'] = df2['headlines'].apply(lambda x: get_total_score(x))


<ipython-input-121-7c4a206284f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['total_score'] = df2['headlines'].apply(lambda x: get_total_score(x))


In [122]:
df2.head(100)

,archiveDate,Urls,headlines,team,total_score
3238,2019-10-14,http://web.archive.org/web/20191014023954/http...,Titans bench Mariota insert Tannehill vs. Broncos,Tennessee Titans,-0.416482
3239,2019-10-14,http://web.archive.org/web/20191014023954/http...,Jerry Jones: Cowboys' freefall 'not just' on G...,Dallas Cowboys,-0.990983
3240,2019-10-14,http://web.archive.org/web/20191014023954/http...,QB Goff: Rams' 3rd straight loss a 'gut check',Los Angeles Rams,-0.981394
3241,2019-10-14,http://web.archive.org/web/20191014023954/http...,Fins bench struggling Rosen turn back to Fitz,Miami Dolphins,-0.669938
3242,2019-10-14,http://web.archive.org/web/20191014040632/http...,Trotter: Vikings offense displays new-found ed...,Minnesota Vikings,0.996184
...,...,...,...,...,...
3353,2019-10-28,http://web.archive.org/web/20191028195507/http...,Cardinals acquire RB Drake from Dolphins,Arizona Cardinals,0.833860
3354,2019-10-28,http://web.archive.org/web/20191028195507/http...,Texans' Watt 'gutted' over season-ending injury,Houston Texans,-0.953000
3355,2019-10-29,http://web.archive.org/web/20191029000309/http...,Pats' Belichick leaves door open to coach into...,New England Patriots,0.713874
3357,2019-10-29,http://web.archive.org/web/20191029000309/http...,Falcons' Quinn: Waste of time to think job status,Atlanta Falcons,-0.929343


In [123]:
# prompt: add df1 to df2 so they are the same dataframe

import pandas as pd
df_scored = pd.concat([df_scored, df2], axis=0)


In [124]:
df_scored.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2695 entries, 19 to 3840
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   archiveDate  2695 non-null   object 
 1   Urls         2695 non-null   object 
 2   headlines    2695 non-null   object 
 3   team         2695 non-null   object 
 4   total_score  2695 non-null   float64
dtypes: float64(1), object(4)
memory usage: 126.3+ KB


In [125]:
# prompt: save dataframe as a csv

df_scored.to_csv('/content/df_scored_5.csv')
